In [ ]:
%matplotlib inline

from datetime import datetime

import keras
import matplotlib.pyplot as plt
import numpy as np
from keras.callbacks import TensorBoard
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Dropout, Flatten
from keras.models import Sequential
from scipy.ndimage import imread
from sklearn.model_selection import train_test_split

from visualize import visualize


Using TensorFlow backend.


In [2]:
# define fixed variables 

batch_size = 256
num_classes = 10
epochs = 10

img_rows, img_cols = 28, 28
input_shape = (img_rows, img_cols, 1)

In [3]:
# Loading image arrays

with np.load("notmnist_small.npz") as f:
        data = f['data']
        labels = f['labels']

x_train, x_test, y_train, y_test = train_test_split(data, labels)

In [4]:
# Convert from 0-255 to 0-1
x_train /= 255
x_test /= 255

In [5]:
# Resize from (m, 28, 28) to (m, 28, 28, 1), since Keras always assumes the 4th
# dimension in case of RGB images, even if we're just doing B/W images.
x_train = x_train.reshape(x_train.shape[0], *input_shape)
x_test = x_test.reshape(x_test.shape[0], *input_shape)

In [6]:
print("Found %s records. Splitting into %s training and %s test records." % (
    x_train.shape[0] + x_test.shape[0],
    x_train.shape[0],
    x_test.shape[0],
))


In [7]:
# Convert from numeric labels to one-hot encoding
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


In [8]:
# This is a simpler model that can easily be trained on a laptop
model = Sequential([
    Conv2D(16, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(64, activation='relu'),
    Dense(num_classes, activation='softmax'),
])

In [9]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [10]:
print("Model Summary:")
print(model.summary())

Model Summary:
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 16)        160       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 16)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 13, 13, 16)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 2704)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                173120    
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_3 (Dense)              (None, 10)                650   

In [11]:
# Declare tensorboard callback
tensorboard = TensorBoard(log_dir='./logs/%s' % datetime.now().strftime('%Y-%m-%d_%H:%M'),
                          histogram_freq=0,
                          write_graph=True,
                          write_images=False)


In [12]:
# Train model on training set
try:
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=(x_test, y_test),
              callbacks=[tensorboard])
except KeyboardInterrupt:
    print("\n\nCaught KeyboardInterrupt, stopping training!")


Train on 14044 samples, validate on 4682 samples
Epoch 1/10


  256/14044 [..............................] - ETA: 3s - loss: 2.2984 - acc: 0.1055

  512/14044 [>.............................] - ETA: 3s - loss: 2.2869 - acc: 0.1230

 1024/14044 [=>............................] - ETA: 2s - loss: 2.2473 - acc: 0.1816

 1536/14044 [==>...........................] - ETA: 2s - loss: 2.2041 - acc: 0.2370

...

12032/14044 [========================>.....] - ETA: 0s - loss: 0.2698 - acc: 0.9223

12544/14044 [=========================>....] - ETA: 0s - loss: 0.2676 - acc: 0.9231

13056/14044 [==========================>...] - ETA: 0s - loss: 0.2676 - acc: 0.9226

13568/14044 [===========================>..] - ETA: 0s - loss: 0.2702 - acc: 0.9216

14044/14044 [==============================] - 2s - loss: 0.2705 - acc: 0.9212 - val_loss: 0.2849 - val_acc: 0.9165

In [13]:
# See how well it did on the test setr
test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=0)

print('Test loss:', test_loss)
print('Test accuracy:', test_accuracy)

Test loss: 0.284903794272
Test accuracy: 0.91648868


In [ ]:
# Visualize the model
visualize(model, "notMNIST_small/G/MTAuMTUgU2F0dXJkYXkgTmlnaHQgQlJLLnR0Zg==.png")

AttributeError: 'str' object has no attribute 'layers'